## DELAYS_SO_FAR

From papers we've read we, as well as general experience, if an airport is experiencing delays on a given morning, this delay will propagate throughout the day to impact later flights. We would like to capture this information explicitly associated by flight **TAIL_NUM**

##### Goal:
For a given flight, represented by TAIL_NUM, figure out the number of times that same TAIL_NUM was delayed previously on that day up to 2 hours before the given flight.

##### Hypothesis:
If I am to fly out of LAX at 7PM to SFO. For my flights given TAIL_NUM, which is a unique identifier for that flight route (origin -> destination) if that TAIL_NUM is experiencing flight delays anytime prior to the time of my flight on that same day, there is a chance that this will cause my flight to be delayed as well.

In [0]:
#Imports
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

import pyspark.sql.functions as f
import pyspark.sql.types as t
from pyspark.sql.functions import isnan, when, count, col

Because our feature is built off of data on a day only up until the 2 hours before a "chosen" flight to predict on, we are able to build this feature safely on data across all years.

In [0]:
data = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/team20SSDK/strategy/data_final")
print("Number of flights (2015 - 2019):  ", data.count())
print("Number of data columns:  ", len(data.columns))

Number of flights (2015 - 2019): 31171199
Number of data columns: 93

#### Running calculations on the whole dataset

In [0]:
keep_cols = ['TAIL_NUM','ORIGIN_UTC','ARR_DEL15','ORIGIN_UTC_ADJ_MAX']
data_trim = data.select(*keep_cols)
print("Number of flights (2015 - 2019):  ", data_trim.count())
print("Number of data columns:  ", len(data_trim.columns))
display(data_trim)

Number of flights (2015 - 2019): 31171199
Number of data columns: 4

TAIL_NUM,ORIGIN_UTC,ARR_DEL15,ORIGIN_UTC_ADJ_MAX
N763AS,2015-02-01T19:28:00.000+0000,0.0,2015-02-01T17:28:00.000+0000
N768AS,2015-05-17T01:23:00.000+0000,0.0,2015-05-16T23:23:00.000+0000
N762AS,2015-05-17T01:40:00.000+0000,1.0,2015-05-16T23:40:00.000+0000
N768AS,2015-06-06T01:23:00.000+0000,1.0,2015-06-05T23:23:00.000+0000
N768AS,2015-07-23T01:20:00.000+0000,0.0,2015-07-22T23:20:00.000+0000
N560AS,2015-08-06T18:03:00.000+0000,0.0,2015-08-06T16:03:00.000+0000
N762AS,2015-10-03T01:24:00.000+0000,0.0,2015-10-02T23:24:00.000+0000
N768AS,2015-10-03T01:45:00.000+0000,0.0,2015-10-02T23:45:00.000+0000
N532AS,2015-10-30T18:14:00.000+0000,1.0,2015-10-30T16:14:00.000+0000
N762AS,2016-05-07T18:15:00.000+0000,0.0,2016-05-07T16:15:00.000+0000


Add a column that represents the beginning of a day so that we can use this in our range query, then register this table for use.

In [0]:
data_trim = data_trim.withColumn("DAY_ZERO", f.date_trunc("day", "ORIGIN_UTC"))
data_trim.registerTempTable('data_trim')

Query to compute row by row feature of the `DELAYS_SO_FAR` for a given `TAIL_NUM` on a given day. 

We choose to order by `ORIGIN_UTC` as the very first flight for a given day was initially not being reflected in our query. By sorting we were able to manual sanity checks that all flights for a given airport and day were being reflected in our output.

Because we are doing a **LEFT_JOIN** the null rows that we end up getting simply reflect the first flight for a day, for which the delays so far will always be **0**. Therefore once our query is complete we are able to do a **fillNa()** and substitute the empty values with **0** as this aligns with the meaning of our metric.

In [0]:
%%time

delays_so_far = spark.sql("""
    SELECT d1.TAIL_NUM, d1.ORIGIN_UTC, sum(d2.ARR_DEL15) as delays_so_far
      FROM data_trim as d1
      LEFT JOIN data_trim d2 
        ON d1.TAIL_NUM = d2.TAIL_NUM 
          AND d2.ORIGIN_UTC BETWEEN d1.DAY_ZERO AND d1.ORIGIN_UTC_ADJ_MAX
      GROUP BY d1.TAIL_NUM, d1.ORIGIN_UTC
  """).orderBy("ORIGIN_UTC")

print(delays_so_far.count())
display(delays_so_far)

31168496

TAIL_NUM,ORIGIN_UTC,delays_so_far
N772UA,2014-12-31T21:10:00.000+0000,null
N76519,2015-01-01T05:54:00.000+0000,null
N607JB,2015-01-01T05:55:00.000+0000,null
N597JB,2015-01-01T06:06:00.000+0000,null
N633NK,2015-01-01T06:55:00.000+0000,null
N653JB,2015-01-01T06:55:00.000+0000,null
N37293,2015-01-01T06:59:00.000+0000,null
N630NK,2015-01-01T07:00:00.000+0000,null
N239JB,2015-01-01T07:07:00.000+0000,null
N528NK,2015-01-01T07:15:00.000+0000,null


CPU times: user 335 ms, sys: 110 ms, total: 445 ms
Wall time: 2min 11s

In [0]:
#Represents the first flight of the day, delays so far is 0.
delays_so_far = delays_so_far.fillna(0, 'delays_so_far')
print("Number of rows  ", delays_so_far.count())
print("Number of data columns:  ", len(delays_so_far.columns))
display(delays_so_far)

Number of rows 31168496
Number of data columns: 3

TAIL_NUM,ORIGIN_UTC,delays_so_far
N772UA,2014-12-31T21:10:00.000+0000,0.0
N76519,2015-01-01T05:54:00.000+0000,0.0
N607JB,2015-01-01T05:55:00.000+0000,0.0
N597JB,2015-01-01T06:06:00.000+0000,0.0
N633NK,2015-01-01T06:55:00.000+0000,0.0
N653JB,2015-01-01T06:55:00.000+0000,0.0
N37293,2015-01-01T06:59:00.000+0000,0.0
N630NK,2015-01-01T07:00:00.000+0000,0.0
N239JB,2015-01-01T07:07:00.000+0000,0.0
N528NK,2015-01-01T07:15:00.000+0000,0.0


In [0]:
delays_so_far.printSchema()

root
-- TAIL_NUM: string (nullable = true)
-- ORIGIN_UTC: timestamp (nullable = true)
-- delays_so_far: double (nullable = false)

We can see that 77.248% of our flights have no delays prior to the flight on the same day, and that it is very rare that we see more than 3-4 delays for a given `TAIL_NUM` prior to that flight on the same day. The output below is a good sanity check, and shows signs of parallel with the imbalance nature of our label class `DEP_DEL15`

In [0]:
d_group_pd = delays_so_far.groupBy("delays_so_far").count().toPandas().sort_values(by=["delays_so_far"])
d_group_pd.head(20)

Out[8]:

,delays_so_far,count
1,0.0,24077110
3,1.0,5065569
6,2.0,1423046
5,3.0,431950
4,4.0,125448
8,5.0,34714
7,6.0,8652
2,7.0,1659
0,8.0,288
10,9.0,37


In [0]:
fig = px.bar(d_group_pd, x='delays_so_far', y='count',
             hover_data=['delays_so_far', 'count'], color='delays_so_far',
             labels={'pop':'Number of Flights '}, height=400)
fig.show()

The reason this final dataset has a lesser number of rows is due to some duplicate `TAIL_NUM`'s in our data. The way our query operates, we end up selecting one of the duplicates that matches the JOIN condition and move forward. This is a mistake in the data as there cannot be identical `TAIL_NUM`'s at the same moment in time. These instances are resolved by the join.

#### Write the feature data, for use in data joining at a later stage.

In [0]:
dbutils.fs.rm("dbfs:/mnt/mids-w261/team20SSDK/strategy/helpers/delays_so_far", True)

#Write cleaned airlines data to our store
delays_so_far.write.parquet("dbfs:/mnt/mids-w261/team20SSDK/strategy/helpers/delays_so_far")